# Copy of XGBoost MLflow 3 Tutorial

Based on [XGBoost MLflow 3 Tutorial](https://docs.databricks.com/aws/en/notebooks/source/mlflow/mlflow-classic-ml-e2e-mlflow-3.html)

In [0]:
%pip install --upgrade -Uqqq mlflow==3.1.1 xgboost optuna uv
%restart_python

In [0]:
# FIXME Create a UC catalog
# registered_model_name = "jacek_laskowski.mlflow.xgboost_regression_model"

dbutils.widgets.text(name="uc_schema", defaultValue="data.jacek_laskowski", label="Unity Catalog's Schema")
uc_schema = dbutils.widgets.get("uc_schema")

dbutils.widgets.text(name="registered_model_name", defaultValue=f"{uc_schema}.xgboost_regression_model", label="Registered Model Name")
registered_model_name = dbutils.widgets.get("registered_model_name")

In [0]:
from typing import Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.api.types import CategoricalDtype
from statsmodels.graphics.mosaicplot import mosaic

import xgboost as xgb

import mlflow
from mlflow.models import infer_signature

In [0]:
mlflow.set_registry_uri("databricks-uc")

In [0]:
def create_regression_data(
    n_samples: int, 
    n_features: int,
    seed: int = 1994,
    noise_level: float = 0.3,
    nonlinear: bool = True
) -> Tuple[pd.DataFrame, pd.Series]:
    """Generates synthetic regression data with interesting correlations for MLflow and XGBoost demonstrations.

    This function creates a DataFrame of continuous features and computes a target variable with nonlinear
    relationships and interactions between features. The data is designed to be complex enough to demonstrate
    the capabilities of XGBoost, but not so complex that a reasonable model can't be learned.

    Args:
        n_samples (int): Number of samples (rows) to generate.
        n_features (int): Number of feature columns.
        seed (int, optional): Random seed for reproducibility. Defaults to 1994.
        noise_level (float, optional): Level of Gaussian noise to add to the target. Defaults to 0.3.
        nonlinear (bool, optional): Whether to add nonlinear feature transformations. Defaults to True.

    Returns:
        Tuple[pd.DataFrame, pd.Series]:
            - pd.DataFrame: DataFrame containing the synthetic features.
            - pd.Series: Series containing the target labels.

    Example:
        >>> df, target = create_regression_data(n_samples=1000, n_features=10)
    """
    rng = np.random.RandomState(seed)
    
    # Generate random continuous features
    X = rng.uniform(-5, 5, size=(n_samples, n_features))
    
    # Create feature DataFrame with meaningful names
    columns = [f"feature_{i}" for i in range(n_features)]
    df = pd.DataFrame(X, columns=columns)
    
    # Generate base target variable with linear relationship to a subset of features
    # Use only the first n_features//2 features to create some irrelevant features
    weights = rng.uniform(-2, 2, size=n_features//2)
    target = np.dot(X[:, :n_features//2], weights)
    
    # Add some nonlinear transformations if requested
    if nonlinear:
        # Add square term for first feature
        target += 0.5 * X[:, 0]**2
        
        # Add interaction between the second and third features
        if n_features >= 3:
            target += 1.5 * X[:, 1] * X[:, 2]
        
        # Add sine transformation of fourth feature
        if n_features >= 4:
            target += 2 * np.sin(X[:, 3])
        
        # Add exponential of fifth feature, scaled down
        if n_features >= 5:
            target += 0.1 * np.exp(X[:, 4] / 2)
            
        # Add threshold effect for sixth feature
        if n_features >= 6:
            target += 3 * (X[:, 5] > 1.5).astype(float)
    
    # Add Gaussian noise
    noise = rng.normal(0, noise_level * target.std(), size=n_samples)
    target += noise
    
    # Add a few more interesting features to the DataFrame
    
    # Add a correlated feature (but not used in target calculation)
    if n_features >= 7:
        df['feature_correlated'] = df['feature_0'] * 0.8 + rng.normal(0, 0.2, size=n_samples)
    
    # Add a cyclical feature
    df['feature_cyclical'] = np.sin(np.linspace(0, 4*np.pi, n_samples))
    
    # Add a feature with outliers
    df['feature_with_outliers'] = rng.normal(0, 1, size=n_samples)
    # Add outliers to ~1% of samples
    outlier_idx = rng.choice(n_samples, size=n_samples//100, replace=False)
    df.loc[outlier_idx, 'feature_with_outliers'] = rng.uniform(10, 15, size=len(outlier_idx))
    
    return df, pd.Series(target, name='target')

In [0]:
def plot_feature_distributions(X: pd.DataFrame, y: pd.Series, n_cols: int = 3) -> plt.Figure:
    """
    Creates a grid of histograms for each feature in the dataset.

    Args:
        X (pd.DataFrame): DataFrame containing synthetic features.
        y (pd.Series): Series containing the target variable.
        n_cols (int): Number of columns in the grid layout.

    Returns:
        plt.Figure: The matplotlib Figure object containing the distribution plots.
    """
    features = X.columns
    n_features = len(features)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if n_rows * n_cols > 1 else [axes]
    
    for i, feature in enumerate(features):
        if i < len(axes):
            ax = axes[i]
            sns.histplot(X[feature], ax=ax, kde=True, color='skyblue')
            ax.set_title(f'Distribution of {feature}')
    
    # Hide any unused subplots
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    fig.suptitle('Feature Distributions', y=1.02, fontsize=16)
    plt.close(fig)
    return fig

def plot_correlation_heatmap(X: pd.DataFrame, y: pd.Series) -> plt.Figure:
    """
    Creates a correlation heatmap of all features and the target variable.

    Args:
        X (pd.DataFrame): DataFrame containing features.
        y (pd.Series): Series containing the target variable.

    Returns:
        plt.Figure: The matplotlib Figure object containing the heatmap.
    """
    # Combine features and target into one DataFrame
    data = X.copy()
    data['target'] = y
    
    # Calculate correlation matrix
    corr_matrix = data.corr()
    
    # Set up the figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Draw the heatmap with a color bar
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap=cmap,
                center=0, square=True, linewidths=0.5, ax=ax)
    
    ax.set_title('Feature Correlation Heatmap', fontsize=16)
    plt.close(fig)
    return fig

def plot_feature_target_relationships(X: pd.DataFrame, y: pd.Series, n_cols: int = 3) -> plt.Figure:
    """
    Creates a grid of scatter plots showing the relationship between each feature and the target.

    Args:
        X (pd.DataFrame): DataFrame containing features.
        y (pd.Series): Series containing the target variable.
        n_cols (int): Number of columns in the grid layout.

    Returns:
        plt.Figure: The matplotlib Figure object containing the relationship plots.
    """
    features = X.columns
    n_features = len(features)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if n_rows * n_cols > 1 else [axes]
    
    for i, feature in enumerate(features):
        if i < len(axes):
            ax = axes[i]
            # Scatter plot with regression line
            sns.regplot(x=X[feature], y=y, ax=ax, 
                       scatter_kws={'alpha': 0.5, 'color': 'blue'}, 
                       line_kws={'color': 'red'})
            ax.set_title(f'{feature} vs Target')
    
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    fig.suptitle('Feature vs Target Relationships', y=1.02, fontsize=16)
    plt.close(fig)
    return fig

def plot_pairwise_relationships(X: pd.DataFrame, y: pd.Series, features: list[str]) -> plt.Figure:
    """
    Creates a pairplot showing relationships between selected features and the target.

    Args:
        X (pd.DataFrame): DataFrame containing features.
        y (pd.Series): Series containing the target variable.
        features (List[str]): List of feature names to include in the plot.

    Returns:
        plt.Figure: The matplotlib Figure object containing the pairplot.
    """
    # Ensure features exist in the DataFrame
    valid_features = [f for f in features if f in X.columns]
    
    if not valid_features:
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, "No valid features provided", ha='center', va='center')
        return fig
    
    # Combine selected features and target
    data = X[valid_features].copy()
    data['target'] = y
    
    # Create pairplot
    pairgrid = sns.pairplot(data, diag_kind="kde", 
                          plot_kws={"alpha": 0.6, "s": 50},
                          corner=True)
    
    pairgrid.fig.suptitle("Pairwise Feature Relationships", y=1.02, fontsize=16)
    plt.close(pairgrid.fig)
    return pairgrid.fig

def plot_boxplots(X: pd.DataFrame, y: pd.Series, n_cols: int = 3) -> plt.Figure:
    """
    Creates a grid of box plots for each feature, with points colored by target value.

    Args:
        X (pd.DataFrame): DataFrame containing features.
        y (pd.Series): Series containing the target variable.
        n_cols (int): Number of columns in the grid layout.

    Returns:
        plt.Figure: The matplotlib Figure object containing the box plots.
    """
    features = X.columns
    n_features = len(features)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if n_rows * n_cols > 1 else [axes]
    
    # Create target bins for coloring
    y_binned = pd.qcut(y, 3, labels=['Low', 'Medium', 'High'])
    
    for i, feature in enumerate(features):
        if i < len(axes):
            ax = axes[i]
            # Box plot for each feature
            sns.boxplot(x=y_binned, y=X[feature], ax=ax)
            ax.set_title(f'Distribution of {feature} by Target Range')
            ax.set_xlabel('Target Range')
    
    # Hide any unused subplots
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    fig.suptitle('Feature Distributions by Target Range', y=1.02, fontsize=16)
    plt.close(fig)
    return fig

def plot_outliers(X: pd.DataFrame, n_cols: int = 3) -> plt.Figure:
    """
    Creates a grid of box plots to detect outliers in each feature.

    Args:
        X (pd.DataFrame): DataFrame containing features.
        n_cols (int): Number of columns in the grid layout.

    Returns:
        plt.Figure: The matplotlib Figure object containing the outlier plots.
    """
    features = X.columns
    n_features = len(features)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if n_rows * n_cols > 1 else [axes]
    
    for i, feature in enumerate(features):
        if i < len(axes):
            ax = axes[i]
            # Box plot to detect outliers
            sns.boxplot(x=X[feature], ax=ax, color='skyblue')
            ax.set_title(f'Outlier Detection for {feature}')
            ax.set_xlabel(feature)
    
    # Hide any unused subplots
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    fig.suptitle('Outlier Detection for Features', y=1.02, fontsize=16)
    plt.close(fig)
    return fig

In [0]:
n_samples = 1000
n_features = 10
X, y = create_regression_data(n_samples=n_samples, n_features=n_features, nonlinear=True)

# Create EDA plots
dist_plot = plot_feature_distributions(X, y)
corr_plot = plot_correlation_heatmap(X, y)
scatter_plot = plot_feature_target_relationships(X, y)
corr_with_target = X.corrwith(y).abs().sort_values(ascending=False)
top_features = corr_with_target.head(4).index.tolist()
pairwise_plot = plot_pairwise_relationships(X, y, top_features)
outlier_plot = plot_outliers(X)

# Configure the XGBoost model
reg = xgb.XGBRegressor(
    tree_method="hist",
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='rmse',
)

# Create train/test split to properly evaluate the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7722)

# Train the model with evaluation
reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

# Generate predictions for residual plot
y_pred = reg.predict(X_test)
residual_plot = plot_boxplots(X, y)

In [0]:
# Incorporate MLflow evaluation
evaluation_data = X_test.copy()
evaluation_data["label"] = y_test

# Log the model and training metadata results
with mlflow.start_run() as run:
    # Extract metrics
    final_train_rmse = np.array(reg.evals_result()["validation_0"]["rmse"])[-1]
    final_test_rmse = np.array(reg.evals_result()["validation_1"]["rmse"])[-1]
    
    # Extract parameters for logging
    feature_map = {key: value for key, value in reg.get_xgb_params().items() if value is not None}

    # Generate a model signature using the infer_signature utility in MLflow
    # A signature is required to register the model to Unity Catalog 
    # so that the model can be used in SQL queries
    signature = infer_signature(X, reg.predict(X))

    # Log parameters
    mlflow.log_params(feature_map)
    
    # Log the model to MLflow and register the model to Unity Catalog
    # All model metrics and parameters will be available in Unity Catalog
    model_info = mlflow.xgboost.log_model(
        xgb_model=reg,
        name="xgboost_regression_model",
        input_example=X.iloc[[0]],
        signature=signature,
        registered_model_name=registered_model_name,
    )

    # Log metrics to the run and model
    mlflow.log_metric("train_rmse", final_train_rmse)
    mlflow.log_metric("test_rmse", final_test_rmse)
    
    # Log feature analysis plots
    # Plots are saved as artifacts in MLflow
    mlflow.log_figure(dist_plot, "feature_distributions.png")
    mlflow.log_figure(corr_plot, "correlation_heatmap.png")
    mlflow.log_figure(scatter_plot, "feature_target_relationships.png")
    mlflow.log_figure(pairwise_plot, "pairwise_relationships.png")
    mlflow.log_figure(outlier_plot, "outlier_detection.png")
    mlflow.log_figure(residual_plot, "feature_boxplots_by_target.png")
        
    # Plot feature importance
    fig, ax = plt.subplots(figsize=(10, 8))
    xgb.plot_importance(reg, ax=ax, importance_type='gain')
    plt.title('Feature Importance')
    plt.tight_layout()
    plt.close(fig)

    mlflow.log_figure(fig, "feature_importance.png")

    # Run MLflow evaluation to generate additional metrics without having to implement them
    mlflow.models.evaluate(
        model=model_info.model_uri, 
        data=evaluation_data, 
        targets="label", 
        model_type="regressor", 
        evaluator_config={"metric_prefix": "mlflow_evaluation_"},
    )
    
    print(f"Model logged: {model_info.model_uri}")
    print(f"Train RMSE: {final_train_rmse:.4f}")
    print(f"Test RMSE: {final_test_rmse:.4f}")

In [0]:
import optuna
from sklearn.metrics import mean_squared_error
import numpy as np
import mlflow
import xgboost as xgb

# Generate training and validation data
n_samples = 2000
n_features = 10

X, y = create_regression_data(n_samples=n_samples, n_features=n_features, nonlinear=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prepare the evaluation data
evaluation_data = X_test.copy()
evaluation_data["label"] = y_test

reg.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)
reg = xgb.XGBRegressor(
    tree_method="hist",
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='rmse',
)

# The objective function defines the search space for the key hyperparameters of the XGBRegressor algorithm.
# Optuna dynamically samples these values, so that each trial tests a different combination of parameters.
def objective(trial):
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "eta": trial.suggest_float("eta", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "tree_method": "hist",
        "objective": "reg:squarederror",
        "eval_metric": "rmse"
    }

    with mlflow.start_run(nested=True):
        mlflow.log_params(param)
        regressor = xgb.XGBRegressor(**param)
        regressor.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

        preds = regressor.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mlflow.log_metric("rmse", rmse)
    
    # Store the model in the trial's `user attributes`
    trial.set_user_attr("model", regressor)
    return rmse

jacek_n_trials = 10

# In the parent run, save the best iteration from the hyperparameter tuning execution
with mlflow.start_run():
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=jacek_n_trials)

    best_trial = study.best_trial
    best_model = best_trial.user_attrs["model"]

    mlflow.log_metric("best_rmse", best_trial.value)
    mlflow.log_params(best_trial.params)

    signature = infer_signature(X_train, best_model.predict(X_test))

    mlflow.xgboost.log_model(
        xgb_model=best_model,     
        name="xgboostoptuna",
        input_example=X_train.iloc[[0]],
        signature=signature,
        model_format="ubj",
        registered_model_name=f"{uc_schema}.xgboostoptuna",
    )

    mlflow.models.evaluate(
        model=model_info.model_uri, 
        data=evaluation_data, 
        targets="label", 
        model_type="regressor", 
        evaluator_config={"metric_prefix": "mlflow_evaluation_"},
    )

    dist_plot = plot_feature_distributions(X_train, y_train)
    corr_plot = plot_correlation_heatmap(X_train, y_train)
    scatter_plot = plot_feature_target_relationships(X_train, y_train)

    # Select a few interesting features for the pairwise plot
    # Choose features with highest correlation with target
    corr_with_target = X_train.corrwith(y_train).abs().sort_values(ascending=False)
    top_features = corr_with_target.head(4).index.tolist()
    pairwise_plot = plot_pairwise_relationships(X, y, top_features)

    # Log the plots associated with the parent run only
    mlflow.log_figure(dist_plot, "feature_distributions.png")
    mlflow.log_figure(corr_plot, "correlation_heatmap.png")
    mlflow.log_figure(scatter_plot, "feature_target_relationships.png")
    mlflow.log_figure(pairwise_plot, "pairwise_relationships.png")
    mlflow.log_figure(outlier_plot, "outlier_detection.png")
    mlflow.log_figure(residual_plot, "feature_boxplots_by_target.png")
        
    # Plot feature importance of the best model only
    fig, ax = plt.subplots(figsize=(10, 8))
    xgb.plot_importance(best_model, ax=ax, importance_type='gain')
    plt.title('Feature Importance')
    plt.tight_layout()
    plt.close(fig)

    mlflow.log_figure(fig, "feature_importance.png")

In [0]:
# Use the `MlflowClient` to access metadata, artifacts, and information about models that are tracked or registered to the model registry.
from mlflow import MlflowClient
client = MlflowClient()

# Set the alias on the desired version. This example uses version 1.
client.set_registered_model_alias(f"{uc_schema}.xgboostoptuna", "best", 1)

In [0]:
model_uri = f"models:/{uc_schema}.xgboostoptuna@best"

mlflow.models.predict(model_uri=model_uri, input_data=X_train, env_manager="uv")

In [0]:
# Load the model and use the model to make predictions locally
loaded_registered_model = mlflow.pyfunc.load_model(model_uri=model_uri)

loaded_registered_model.predict(X_train)

In [0]:
# Convert the training data into a Spark DataFrame.
X_spark = spark.createDataFrame(X_train)

# Create a Spark UDF to apply the model to the Spark DataFrame.
# Note that `model_uri` is defined based on a model alias, ensuring that you always load the current, approved version.
prediction_udf = mlflow.pyfunc.spark_udf(
    spark,
    model_uri=model_uri,
)

# Apply the Spark UDF to the DataFrame. This performs batch predictions across all rows in a distributed manner. 
X_spark = X_spark.withColumn("prediction", prediction_udf(*X_train.columns))

# Display the resulting DataFrame. 
display(X_spark)

## Extras

In [0]:
display(X_spark)

In [0]:
type(X_spark)

In [0]:
# X_spark is a features table
X_spark.limit(10).write.mode("overwrite").saveAsTable(f"{uc_schema}.features")

In [0]:
dbutils.notebook.exit("Stop here")


### Create Lakeflow Transformation

Add a new transformation as described in [Use MLflow models in Lakeflow Declarative Pipelines](https://docs.databricks.com/aws/en/dlt/transform#use-mlflow-models-in-lakeflow-declarative-pipelines)

In [0]:
import dlt
import mlflow

uc_schema = "data.jacek_laskowski"
features_table_name = f"{uc_schema}.features"

@dlt.view
def features():
    # Uncomment to make sure that you're on MLflow 3.1.1 or higher
    # features_table = spark.createDataFrame(data=[{"mlflow_version": mlflow.__version__}])

    # If the above worked, use the features table that we registered in the other meetup notebook
    features_table = spark.table(features_table_name).limit(5)

    return features_table


@dlt.table
def predictions():

    model_uri = f"models:/{uc_schema}.xgboostoptuna@best"
    mlflow.pyfunc.get_model_dependencies(model_uri)
    prediction_udf = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

    features_table = dlt.read("features")

    # Return features_table with predictions
    return (
        features_table
        .withColumn("prediction", prediction_udf(*features_table.columns))
    )



### ModuleNotFoundError: No module named 'flask'

Make sure that `mlflow==3.1.1` is defined as a dependency in `Settings`

<br>

```
ModuleNotFoundError: Traceback (most recent call last):
  File "/Workspace/Users/jlaskowski@pl.imshealth.com/lakeflow_demo/transformations/mlflow_demo.py", cell 1, line 10, in x_spark_predictions
    model_udf = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/databricks/python_shell/lib/dbruntime/MLWorkloadsInstrumentation/_spark_udf.py", line 56, in patched_spark_udf
    return original_spark_udf(
           ^^^^^^^^^^^^^^^^^^^

  File "/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py", line 1827, in spark_udf
    pyfunc_backend = get_flavor_backend(
                     ^^^^^^^^^^^^^^^^^^^

  File "/databricks/python/lib/python3.12/site-packages/mlflow/models/flavor_backend_registry.py", line 48, in get_flavor_backend
    flavor_name, flavor_backend = _get_flavor_backend_for_local_model(model, **kwargs)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/databricks/python/lib/python3.12/site-packages/mlflow/models/flavor_backend_registry.py", line 23, in _get_flavor_backend_for_local_model
    from mlflow.pyfunc.backend import PyFuncBackend

  File "/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/backend.py", line 17, in <module>
    from mlflow.pyfunc import (

  File "/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py", line 6, in <module>
    from mlflow.pyfunc.scoring_server import _predict

  File "/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/scoring_server/__init__.py", line 24, in <module>
    import flask

ModuleNotFoundError: No module named 'flask'
```

### ModuleNotFoundError: No module named 'xgboost'

Make sure that `xgboost` is defined as a dependency in `Settings`

<br>

```
ModuleNotFoundError: Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py", line 2731, in udf
    loaded_model = mlflow.pyfunc.load_model(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/tracing/provider.py", line 442, in wrapper
    except MlflowTracingException as e:

  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py", line 1187, in load_model
    _clear_dependencies_schemas()

  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py", line 1187, in load_model
    _clear_dependencies_schemas()

  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/xgboost/__init__.py", line 330, in _load_pyfunc
    return _XGBModelWrapper(_load_model(path))
                            ^^^^^^^^^^^^^^^^^

  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/xgboost/__init__.py", line 319, in _load_model
    model = _get_class_from_string(model_class)()
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-6fe1dd7c-f554-48ec-ae05-06df73d3064e/lib/python3.12/site-packages/mlflow/utils/class_utils.py", line 6, in _get_class_from_string
    return getattr(importlib.import_module(module), class_name)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1310, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1324, in _find_and_load_unlocked


ModuleNotFoundError: No module named 'xgboost'
```


![databricks-machine-learning-model-deployment-lesson.png](./databricks-machine-learning-model-deployment-lesson.png)

## Follow-up Ideas

1. Use [databricks-feature-engineering](https://pypi.org/project/databricks-feature-engineering/) to access features in Databricks online table
1. https://learn.microsoft.com/en-us/azure/databricks/mlflow/deployment-job
1. https://learn.microsoft.com/en-us/azure/databricks/mlflow3/genai/prompt-version-mgmt/prompt-registry/create-and-edit-prompts
1. https://learn.microsoft.com/en-us/azure/databricks/dlt/dlt-multi-file-editor
1. https://learn.microsoft.com/en-us/azure/databricks/mlflow/databricks-autologging
1. Traffic Splitting
    * Deploy multiple models behind the same endpoint

# Machine Learning Practitioner Learning Plan

➡️ [Machine Learning Practitioner Learning Plan](https://partner-academy.databricks.com/learn/learning-plans/11/machine-learning-practitioner-learning-plan)


## Machine Learning Model Deployment course Completed

![Machine_Learning_Model_Deployment_course_completed](Machine_Learning_Model_Deployment_course_completed.png)